### Loading Packages and Reading Table Data
Import packages and request wikipedia url. Find all values in the wikipedia table in-between th/td and assume these are all the table values we are interested in. This assumption has been double checked.

In [9]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(wiki_url.text, 'lxml')
for items in soup.find_all("table", class_="wikitable sortable"):
    data = [' '.join(item.text.split()) for item in items.find_all(['th','td'])]
table_data = np.reshape(data, (-1,3)) 
table_data

array([['Postcode', 'Borough', 'Neighbourhood'],
       ['M1A', 'Not assigned', 'Not assigned'],
       ['M2A', 'Not assigned', 'Not assigned'],
       ['M3A', 'North York', 'Parkwoods'],
       ['M4A', 'North York', 'Victoria Village'],
       ['M5A', 'Downtown Toronto', 'Harbourfront'],
       ['M5A', 'Downtown Toronto', 'Regent Park'],
       ['M6A', 'North York', 'Lawrence Heights'],
       ['M6A', 'North York', 'Lawrence Manor'],
       ['M7A', "Queen's Park", 'Not assigned'],
       ['M8A', 'Not assigned', 'Not assigned'],
       ['M9A', 'Etobicoke', 'Islington Avenue'],
       ['M1B', 'Scarborough', 'Rouge'],
       ['M1B', 'Scarborough', 'Malvern'],
       ['M2B', 'Not assigned', 'Not assigned'],
       ['M3B', 'North York', 'Don Mills North'],
       ['M4B', 'East York', 'Woodbine Gardens'],
       ['M4B', 'East York', 'Parkview Hill'],
       ['M5B', 'Downtown Toronto', 'Ryerson'],
       ['M5B', 'Downtown Toronto', 'Garden District'],
       ['M6B', 'North York', 'Glencairn'

### Read data into Pandas Dataframe
read the table_data into a panda dataframe. Replace "not assigned" neighborhood with corresponding value from Boroughs. Remove rows with "not assigned" boroughs.
Group and sum strings by PostalCode/Borough and make sure they are separated by comma. At last find shape of new cleaned datafram.

In [11]:
df = pd.DataFrame()
df['PostalCode'] = table_data[1:,0]
df['Borough'] = table_data[1:,1]
df['Neighborhood'] = table_data[1:,2]
df['Neighborhood'].replace("Not assigned", df['Borough'], inplace=True)
df['Borough'].replace("Not assigned", pd.np.nan, inplace=True)
df.dropna(axis = 0, subset=["Borough"], inplace=True)
df.reset_index()

for i in range(0,len(df['Neighborhood'])):
    df.Neighborhood.iloc[i] = df.Neighborhood.iloc[i] + ", "
    
df = df.groupby(["PostalCode", "Borough"], as_index = False, sort = False).sum()
for i in range(0,len(df['Neighborhood'])):
    if df.Neighborhood.iloc[i].endswith(", "):
        df.Neighborhood.iloc[i] = df.Neighborhood.iloc[i][:-2]
        
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [12]:
df.shape

(103, 3)

### Assign Longitudes/Lattitude to Postal Codes
Import data from .csv file, and merge the two data sets on the postal code.

In [13]:
data = pd.read_csv('http://cocl.us/Geospatial_data')

In [14]:
df['Lattitude'] = "NAN"
df['Longitude'] = "NAN"
for i in range(0, len(df['PostalCode'])):
    for ii in range(0, len(data)):
        if df['PostalCode'].iloc[i] == data.iloc[ii][0]:
            df['Lattitude'].iloc[i] = float(data.iloc[ii][1])
            df['Longitude'].iloc[i] = float(data.iloc[ii][2])
            break
df

,PostalCode,Borough,Neighborhood,Lattitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
5,M9A,Etobicoke,Islington Avenue,43.6679,-79.5322
6,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
7,M3B,North York,Don Mills North,43.7459,-79.3522
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7064,-79.3099
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6572,-79.3789


### Visualization

Visualize only borough with Toronto in them.

In [15]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

for i in range(0,len(df)):
    if "Toronto" nothttps://dataplatform.cloud.ibm.com/data/jupyter2/runtimeenv2/v1/wdpx/service/notebook/conda2x4d9627110b3f14cfc83bcc803ea239d65/dsxjpy/dded3b111c2248748711f2e0af8908a960fd32ff81d506874ba16ca6d27c85a032d73d3162155e40276ce416ddb270b93d0e8afa3a2a59/container/notebooks/80480f12-6edf-4b68-ba43-62cf4ddb71bc?project=d9627110-b3f1-4cfc-83bc-c803ea239d65&api=v2&env=a#Visualization in df['Borough'].iloc[i]:
        df['Borough'].iloc[i] = pd.np.nan
df.dropna(axis = 0, subset=["Borough"], inplace=True)

map_clusters = folium.Map(location=[df["Lattitude"].mean(), df["Longitude"].mean()], zoom_start=12)
for lat, lon in zip(df['Lattitude'], df['Longitude']):
   folium.CircleMarker(
        [lat, lon],
        radius=5,
       fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.14 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.67 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  38.08 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.42 MB/s


### Construct Clustering Model and Compare with Real Districts

We know there are four district with Toronto in its name, lets just for fun and as an illustration show how accurate a k mean clustering algorithm with k = 4 can predict the shape of these districts

In [16]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=0).fit(df.iloc[:,3:5].values.reshape(-1,2))
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

plot the results, see how the clustering algorihm cluster in east, west, center and north

In [17]:
import matplotlib.cm as cm
import matplotlib.colors as colors


map_clusters = folium.Map(location=[df["Lattitude"].mean(), df["Longitude"].mean()], zoom_start=12)

# set color scheme for the clusters
x = np.arange(19)
ys = [i+x+(i*x)**2 for i in range(19)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lon, cluster in zip(df['Lattitude'], df['Longitude'], kmeans.labels_):
   folium.CircleMarker(
        [lat, lon],
       color='black',
        radius=7,
       fill=True,
       fill_color=rainbow[4*cluster+1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

Now figure out how much the the clustering corresponds to the four real districts

In [18]:
yhat = []
for i in range(0,len(df)):
    if "Downtown" in df["Borough"].iloc[i]:
        yhat.append(0)
    elif "West" in df["Borough"].iloc[i]:
        yhat.append(2)
    elif "East" in df["Borough"].iloc[i]:
        yhat.append(3)
    else:
        yhat.append(1)

In [19]:
accuracy = sum(yhat == kmeans.labels_)/len(yhat)
accuracy

0.94736842105263153

almost 95 percent accurate in predicting the district as k mean clusters!